In [1]:
import os
import sys
import glob
import scipy
import sklearn
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_rows', 100)
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['pdf.fonttype'] = 42
pd.set_option('display.max_columns', 100)

sns.set_palette("Set2")


In [2]:
data_loc = '/home/mattolm/user_data/IndianMicrobiome/tables/'
Pdb = pd.read_csv(data_loc + 'IndianMicrobiome_basicinfo_v3.csv')
Pdb


sample                                              read1  \
0      AK_SR_1  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
1      AK_SR_2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
2      AK_SR_2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
3    AK_SR_2.2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
4    AK_SR_2.2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
..         ...                                                ...   
109      TS8_B  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
110     TS12_A  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
111     TS12_B  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
112     TS14_A  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
113     TS14_B  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   

                                                 read2            library_key  \
0    s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A2_AK-SR-1   
1    s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A4_AK-SR-2   
2    s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A4_AK-SR-2   
3    s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  LibPlate-D2_AK-SR-2-2   
4    s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  LibPlate-D2_AK-SR-2-2   
..                                                 ...                    ...   
109  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...      LibPlate-P4_TS8-B   
110  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...     LibPlate-B6_TS12-A   
111  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...     LibPlate-D6_TS12-B   
112  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...     LibPlate-F6_TS14-A   
113  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...     LibPlate-H6_TS14-B   

     DNA_concentration sample_type fecal_sample     pairs       bases  \
0                 51.5       fecal      AK_SR_1  24276525  6938547858   
1                 28.7       fecal      AK_SR_2  25189547  7192762783   
2                 28.7       fecal      AK_SR_2  25189547  7192762783   
3                 92.8       fecal      AK_SR_2  11357247  3183231459   
4                 92.8       fecal      AK_SR_2  11357247  3183231459   
..                 ...         ...          ...       ...         ...   
109                NaN  enrichment      AK_SW_8   6058407  1715849529   
110                NaN  enrichment     AK_SW_12  10370419  2951418443   
111                NaN  enrichment     AK_SW_12   4568279  1283287093   
112                NaN  enrichment     AK_SW_14   2789460   786041399   
113                NaN  enrichment     AK_SW_14   6605728  1864917112   

     raw_pairs    raw_bases          RL  sample_name individual_id sample_id  \
0     40041788  11692202096  142.906529          SR1           SR1   st.t.R1   
1     30917233   9027832036  142.772770        SR2-2         SR2.2   st.R2.2   
2     30917233   9027832036  142.772770        SR2-2         SR2.2   st.R2.2   
3     20045323   5853234316  140.140980        SR2-2         SR2.2   st.R2.2   
4     20045323   5853234316  140.140980        SR2-2         SR2.2   st.R2.2   
..         ...          ...         ...          ...           ...       ...   
109   10693765   3122579380  141.608968   AK-SW8_S59            W8     st.W8   
110   15786166   4609560472  142.299865  AK-SW12_S62           W12    st.W12   
111    8792416   2567385472  140.456296  AK-SW12_S62           W12    st.W12   
112    3550125   1036636500  140.894904  AK-SW14_S63           W14    st.W14   
113   12453511   3636425212  141.159090  AK-SW14_S63           W14    st.W14   

    Gender   Age    BMI  Tribe  Village      Region  Terrain  \
0        F  31.0    NaN  Kabui  Longsai  North-East    Hills   
1        M  55.0    NaN  Kabui  Longsai  North-East    Hills   
2        M  55.0    NaN  Kabui  Longsai  North-East    Hills   
3        M  55.0    NaN  Kabui  Longsai  North-East    Hills   
4        M  55.0    NaN  Kabui  Longsai  North-E

## Make cross-mapping commands

In [3]:
Pdb.head()

sample                                              read1  \
0    AK_SR_1  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
1    AK_SR_2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
2    AK_SR_2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
3  AK_SR_2.2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
4  AK_SR_2.2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   

                                               read2            library_key  \
0  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A2_AK-SR-1   
1  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A4_AK-SR-2   
2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A4_AK-SR-2   
3  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  LibPlate-D2_AK-SR-2-2   
4  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  LibPlate-D2_AK-SR-2-2   

   DNA_concentration sample_type fecal_sample     pairs       bases  \
0               51.5       fecal      AK_SR_1  24276525  6938547858   
1               28.7       fecal      AK_SR_2  25189547  7192762783   
2               28.7       fecal      AK_SR_2  25189547  7192762783   
3               92.8       fecal      AK_SR_2  11357247  3183231459   
4               92.8       fecal      AK_SR_2  11357247  3183231459   

   raw_pairs    raw_bases          RL sample_name individual_id sample_id  \
0   40041788  11692202096  142.906529         SR1           SR1   st.t.R1   
1   30917233   9027832036  142.772770       SR2-2         SR2.2   st.R2.2   
2   30917233   9027832036  142.772770       SR2-2         SR2.2   st.R2.2   
3   20045323   5853234316  140.140980       SR2-2         SR2.2   st.R2.2   
4   20045323   5853234316  140.140980       SR2-2         SR2.2   st.R2.2   

  Gender   Age  BMI  Tribe  Village      Region Terrain  \
0      F  31.0  NaN  Kabui  Longsai  North-East   Hills   
1      M  55.0  NaN  Kabui  Longsai  North-East   Hills   
2      M  55.0  NaN  Kabui  Longsai  North-East   Hills   
3      M  55.0  NaN  Kabui  Longsai  North-East   Hills   
4      M  55.0  NaN  Kabui  Longsai  North-East   Hills   

                                            assembly  assembly_length  \
0  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...      105207200.0   
1  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...      109645423.0   
2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...      109645423.0   
3  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...       64752090.0   
4  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...       64752090.0   

   assembly_N50  
0       41150.0  
1       44744.0  
2       44744.0  
3        9211.0  
4        9211.0

In [83]:
rdb = py_sa.load_running_aegea()

bucket = "s3://czbiohub-microbiome/"
for tribe, db in Pdb.groupby('Tribe'):
    
    # Do the single assemblies
    for i, row in db.iterrows():
        
        # Get the destination
        dest_name = '/'.join(row['assembly'].split('/')[:6]) + '/07_MAPPING/'
        
        # Get the index path and name
        ind_path = '/'.join(row['assembly'].split('/')[:-1])
        ind_name = row['assembly'].split('/')[-1] #+ '.bt2'
        index_loc = ind_path + '/' + ind_name
        
        for j, srow in db.iterrows():
            
            # Get the name
            filename = "{0}-vs-{1}".format(ind_name, srow['sample'])
            
            # Get the reads
            r1 = srow['read1']
            r2 = srow['read2']
            
            cmd, result = make_mapping_command(r1, r2, row['assembly'], dest_name, index_location=index_loc,
                                               ram=16000, cores=8, ret_result=True, output_type='SUMMARY')
            
            submit_aegea_job(cmd, result, rdb=rdb, verbose=True)


Running showq...
Showq succeeded
0 aegea jobs are currently running
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_10/07_MAPPING/SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_10.fasta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SK_10_R1.fastq.gz.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_10/07_MAPPING/SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_10.fasta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SK_19_R1.fastq.gz.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_10/07_MAPPING/SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_10.fasta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SK_32_R1.fastq.gz.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_10/07_MAPPING/SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_10.fasta-vs-NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SK_35_R1.fastq.gz.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_10/07_MAPPING/SCAFFOLDS_1500_MERGED_K77_METASPADES__A

## Parse depth tables

In [89]:
def make_metabat_depth(dbs, SAMPLE='sample'):
    '''
    Example header:
    $ aws s3 cp s3://czbiohub-microbiome/Sonnenburg_Lab/Hadza_Nepal_Metagenomics/FINAL_PROJECT_DATA/Pilot/Pilot_MoBio_Fiber_N_24_7019/08_BINNING/DEPTH_SUBJECT_CONTIGS_1500_ASSEMBLY_K77_MERGED__Pilot_MoBio_Fiber_N_24_7019.txt ./

    $ head DEPTH_SUBJECT_CONTIGS_1500_ASSEMBLY_K77_MERGED__Pilot_MoBio_Fiber_N_24_7019.txt

    contigName      contigLen       totalAvgDepth   Pilot_MoBio_Fiber_A_5_1019__MERGED__on__CONTIGS_1500_ASSEMBLY_K77_MERGED__Pilot_MoBio_Fiber_N_24_7019.bam       Pilot_MoBio_Fiber_A_5_1019__MERGED__on__CONTIGS_1500_ASSEMBLY_K77_MERGED__Pilot_MoBio_Fiber_N_24_7019.bam-var        Pilot_MoBio_Fiber_N_24_7019__MERGED__on__CONTIGS_1500_ASSEMBLY_K77_MERGED__Pilot_MoBio_Fiber_N_24_7019.bam   Pilot_MoBio_Fiber_N_24_7019__MERGED__on__CONTIGS_1500_ASSEMBLY_K77_MERGED__Pilot_MoBio_Fiber_N_24_7019.bam-var
    NODE_1_length_771540_cov_117.928032     771540  497.052 292.721 995.536 204.331 634.503
    NODE_2_length_721281_cov_20.320710      721281  45.5879 10.3349 12.1768 35.253  47.4697
    NODE_3_length_532809_cov_28.178136      532809  303.212 254.23  595.608 48.9825 66.2195
    NODE_4_length_528096_cov_22.580496      528096  50.2436 11.0432 12.814  39.2004 49.4816
    NODE_5_length_519115_cov_90.726754      519115  388.303 229.386 716.021 158.917 465.299
    NODE_6_length_510507_cov_83.697102      510507  298.324 152.322 365.336 146.002 257.475
    NODE_7_length_494467_cov_14.341979      494467  31.0065 6.83192 153.226 24.1746 46.928
    NODE_8_length_460014_cov_38.454184      460014  216.169 149.858 236.12  66.3111 92.877
    NODE_9_length_424083_cov_27.079862      424083  295.857 249.084 626.746 46.7723 58.5462
    '''
    # Validate
    contigs = set()
    mdb = None
    for i, db in enumerate(dbs):
        sample = db[SAMPLE].tolist()[0]
        scontigs = set(db['contigName'].tolist())
        tmdb = db[['contigName', 'contigLen', 'output.bam', 'output.bam-var']].rename(columns={
                'output.bam':sample, 'output.bam-var':sample + '-var'})
        
        if i == 0:
            contigs = scontigs
            mdb = tmdb
        else:
            assert contigs == scontigs
            mdb = pd.merge(mdb, tmdb, on=['contigName', 'contigLen'])
            
    # Add average depth
    cols = list(mdb.columns)
    assert len(cols) == (len(dbs) * 2) + 2
    data_cols = [c for c in cols if (('-var' not in c) & (c not in ['contigName', 'contigLen']))]
    assert len(data_cols) == len(dbs)
    mdb['totalAvgDepth'] = sum([mdb[x] for x in data_cols])
        
    # Put it in the right order
    cols.insert(2, 'totalAvgDepth')
    mdb = mdb[cols]
        
    return mdb

def save_s3_table(df, bucket, key):
    '''
    https://stackoverflow.com/questions/38154040/save-dataframe-to-csv-directly-to-s3-python
    '''
    from io import StringIO # python3; python2: BytesIO 
    import boto3

    csv_buffer = StringIO()
    df.to_csv(csv_buffer, sep='\t', index=False)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, key).put(Body=csv_buffer.getvalue())


In [ ]:
bucket = "s3://czbiohub-microbiome/"
s3_bucket = 'czbiohub-microbiome'

table = defaultdict(list)
for tribe, db in Pdb.groupby('Tribe'):
    
    # Do the single assemblies
    for i, row in db.iterrows():
        
        # Get the report location
        dest_name = '/'.join(row['assembly'].split('/')[:6]) + '/07_MAPPING'
        
        # Get the fasta location
        ind_path = '/'.join(row['assembly'].split('/')[:-1])
        ind_name = row['assembly'].split('/')[-1] #+ '.bt2'
        index_loc = ind_path + '/' + ind_name
        
        # Get all the reports
        dbs = []
        for j, srow in db.iterrows():
            
            # Get the name
            filename = "{0}-vs-{1}".format(ind_name, os.path.basename(srow['read1']))
            search_name = dest_name + '/' + filename
            file = [f for f in get_matching_s3_keys(s3_bucket, search_name.replace('s3://czbiohub-microbiome/', ''))]
            assert len(file) == 1, search_name
            file = file[0]
            
            # Load the report
            rdb = load_coverage_report(s3_bucket, file)
            rdb = rdb.rename(columns={c: 'output.bam' if c.endswith('.bam') else 'output.bam-var' if c.endswith('bam-var') else c for c in rdb.columns  })
            rdb['sample'] = srow['sample']
            dbs.append(rdb)
            
        
        # Make combined table
        assert len(db) == len(dbs)
        mdb = make_metabat_depth(dbs)
        
        # Save combined table
        save_loc =  '/'.join(row['assembly'].split('/')[3:6]) + '/08_BINNING/TribeDepthTable.txt'
        save_s3_table(mdb, s3_bucket, save_loc)

        print("Coverage table made for assembly {0} at {2}{1}".format(row['sample'], save_loc, bucket))


Coverage table made for assembly AK_SK_10 at s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_10/08_BINNING/TribeDepthTable.txt
Coverage table made for assembly AK_SK_19 at s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_19/08_BINNING/TribeDepthTable.txt
Coverage table made for assembly AK_SK_32 at s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_32/08_BINNING/TribeDepthTable.txt
Coverage table made for assembly AK_SK_35 at s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_35/08_BINNING/TribeDepthTable.txt
Coverage table made for assembly AK_SK_44 at s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_44/08_BINNING/TribeDepthTable.txt
Coverage table made for assembly AK_SK_8 at s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_8/08_BINNING/TribeDepthTable.txt
Coverage table made for assembly AK_SK_40 at s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_40/08_BINNING/TribeDepthTable.txt
Coverage

## Make binning command

In [95]:
def make_binning_command(assembly_folder, report_loc, assembly_name, s3_store_folder, **kwargs):
    '''
    Generate a binning command
    '''
    # Get s3 arguments
    bucket_id = kwargs.get('bucket_id', "czbiohub-microbiome")
    image = kwargs.get('image', 'sonnenburglab/genome_binning:latest')
    ret_result = kwargs.get('ret_result', False)
    
    # Get aegea arguments
    ram = kwargs.get('ram', 16000)
    cores = kwargs.get('cores', 8)
    
    # Make the command
    cmd = f"""
    export ASSEMBLY_PATH={assembly_folder};
    export DEPTH_PATH={report_loc};
    export BIN_PATH={s3_store_folder};
    export BINNING_TYPE=SUBJECTMAPPING;
    export RENAME_CONTIGS=FALSE;
    export coreNum={cores};
    export mem_mb={ram};
    export ASSEMBLY_NAME={assembly_name};
    source depthdone_metabat_binning_and_mash.sh
    """.replace('\n', ' ')
    
    
    cmd = ' '.join(cmd.split()) # This removes variable length whitespace

    result = f"{s3_store_folder}/logs/VERSION__genome_binning.txt"
    
    # Wrap in aegea
    aegea_cmd = "aegea batch submit --queue sonnenburg__spot100 --image {3} --storage /mnt=500 --vcpus {0} --memory {1} --command=\"{2}\" &>> AEGEA.log".format(
        cores, ram, cmd, image)
    
    
    if ret_result:
        return aegea_cmd, result
    else:
        return aegea_cmd
    

In [103]:
rdb = load_running_aegea()

for i, row in Pdb.iterrows():
    if row['assembly'] != row['assembly']:
        continue
    
#     if row['sample'] != 'AK_SK_10':
#         continue
    
    # Get the report location
    rloc = bucket + '/'.join(row['assembly'].split('/')[3:6]) + '/08_BINNING/TribeDepthTable.txt'
    #assert rloc[25:] == a2d[row['SampleName']], [rloc[25:], a2d[row['SampleName']]]
    
    # Get the assembly location
    assloc = row['assembly']
    ass_folder = '/'.join(assloc.split('/')[:-1])
    ass_name = assloc.split('/')[-1]
    
    # Get the output path
    bpath = bucket + '/'.join(row['assembly'].split('/')[3:6]) + '/08_BINNING'
    
    # Make command
    cmd, result = make_binning_command(ass_folder, rloc, ass_name, bpath, ret_result=True)
    submit_aegea_job(cmd, result, verbose=True, rdb=rdb)



Running showq...
Showq succeeded
0 aegea jobs are currently running
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_1/08_BINNING/logs/VERSION__genome_binning.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_2/08_BINNING/logs/VERSION__genome_binning.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_2/08_BINNING/logs/VERSION__genome_binning.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_2.2/08_BINNING/logs/VERSION__genome_binning.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_2.2/08_BINNING/logs/VERSION__genome_binning.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_3/08_BINNING/logs/VERSION__genome_binning.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_4/08_BINNING/logs/VERSION__genome_binning.txt already exists
s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_

In [87]:
cmd

'aegea batch submit --queue sonnenburg__spot100 --image sonnenburglab/genome_binning:latest --storage /mnt=500 --vcpus 8 --memory 16000 --command="export ASSEMBLY_PATH=s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_10/06_ASSEMBLY; export DEPTH_PATH=s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_10/08_BINNING/TribeDepthTable.txt; export BIN_PATH=s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SK_10/08_BINNING; export BINNING_TYPE=SUBJECTMAPPING; export RENAME_CONTIGS=FALSE; export coreNum=8; export mem_mb=16000; export ASSEMBLY_NAME=SCAFFOLDS_1500_MERGED_K77_METASPADES__AK_SK_10.fasta; source depthdone_metabat_binning_and_mash.sh" &>> AEGEA.log'